# 請結合前面的知識與程式碼，比較不同的 optimizer 與 learning rate 組合對訓練的結果與影響
### 常見的 optimizer 包含
* SGD
* RMSprop
* AdaGrad
* Adam

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
def build_mlp(input_shape, output_units=10):
    input_layer = keras.layers.Input(input_shape)
    X = keras.layers.Dense(units=512, activation='relu')(input_layer)
    X = keras.layers.Dropout(0.2)(X)
    X = keras.layers.Dense(units=256, activation='relu')(X)
    X = keras.layers.Dense(units=256, activation='relu')(X)
    X = keras.layers.Dropout(0.2)(X)
    X = keras.layers.Dense(units=128, activation='relu')(X)
    X = keras.layers.Dense(units=64, activation='relu')(X)
    X = keras.layers.Dense(units=32, activation='relu')(X)
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(X)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [7]:
## 超參數設定
optim_type = ['sgd', 'adam','rmsprop']
lr_list  = [0.1,0.01,0.001]
EPOCHS = 50
BATCH_SIZE = 256


model-sks_lr-1


In [10]:
results = {}
for lr in lr_list:
    for optim in optim_type:
        keras.backend.clear_session()

        if optim == 'sgd':
            optimizer = keras.optimizers.SGD(lr=lr, nesterov=True, momentum=0.9)
        elif optim == 'adam':
            optimizer = keras.optimizers.Adam(lr=lr)
        else:
            optimizer = keras.optimizers.RMSprop(lr=lr)
        
        print(f"Run with optimizer = {optim}, lr = {lr}")
        model = build_mlp(input_shape=x_train.shape[1:])
        model.summary()
        model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)
        
        model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    # Collect results
        train_loss = model.history.history["loss"]
        valid_loss = model.history.history["val_loss"]
        train_acc = model.history.history["acc"]
        valid_acc = model.history.history["val_acc"]
    
        exp_name_tag = f"model-{str(optim)}_lr-{str(lr)}" 
        results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}        

Run with optimizer = sgd, lr = 0.1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128) 

50000/50000 [==============================] - 12s 239us/step - loss: 1.4412 - acc: 0.4878 - val_loss: 1.4871 - val_acc: 0.4761
Epoch 48/50
50000/50000 [==============================] - 12s 233us/step - loss: 1.4331 - acc: 0.4899 - val_loss: 1.4788 - val_acc: 0.4615
Epoch 49/50
50000/50000 [==============================] - 12s 233us/step - loss: 1.4338 - acc: 0.4935 - val_loss: 1.4615 - val_acc: 0.4832
Epoch 50/50
50000/50000 [==============================] - 12s 232us/step - loss: 1.4279 - acc: 0.4956 - val_loss: 1.4705 - val_acc: 0.4802
Run with optimizer = adam, lr = 0.1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout) 

50000/50000 [==============================] - 15s 303us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 43/50
50000/50000 [==============================] - 15s 296us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 44/50
50000/50000 [==============================] - 15s 296us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 45/50
50000/50000 [==============================] - 15s 300us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 46/50
50000/50000 [==============================] - 15s 297us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 47/50
50000/50000 [==============================] - 15s 295us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 48/50
50000/50000 [==============================] - 15s 296us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 49/50
50000/

50000/50000 [==============================] - 13s 262us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 38/50
50000/50000 [==============================] - 13s 260us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 39/50
50000/50000 [==============================] - 13s 260us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 40/50
50000/50000 [==============================] - 13s 263us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 41/50
50000/50000 [==============================] - 13s 258us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 42/50
50000/50000 [==============================] - 13s 260us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 43/50
50000/50000 [==============================] - 13s 260us/step - loss: 14.5063 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 44/50
50000/

Epoch 33/50
50000/50000 [==============================] - 12s 234us/step - loss: 1.1699 - acc: 0.5800 - val_loss: 1.3504 - val_acc: 0.5236
Epoch 34/50
50000/50000 [==============================] - 12s 233us/step - loss: 1.1576 - acc: 0.5860 - val_loss: 1.3116 - val_acc: 0.5325
Epoch 35/50
50000/50000 [==============================] - 12s 232us/step - loss: 1.1508 - acc: 0.5847 - val_loss: 1.3194 - val_acc: 0.5305
Epoch 36/50
50000/50000 [==============================] - 12s 233us/step - loss: 1.1360 - acc: 0.5921 - val_loss: 1.3092 - val_acc: 0.5383
Epoch 37/50
50000/50000 [==============================] - 12s 232us/step - loss: 1.1281 - acc: 0.5935 - val_loss: 1.3115 - val_acc: 0.5372
Epoch 38/50
50000/50000 [==============================] - 12s 232us/step - loss: 1.1234 - acc: 0.5966 - val_loss: 1.3246 - val_acc: 0.5319
Epoch 39/50
50000/50000 [==============================] - 12s 237us/step - loss: 1.1150 - acc: 0.5989 - val_loss: 1.3263 - val_acc: 0.5308
Epoch 40/50
50000/50

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

"""
將實驗結果繪出
"""